In [21]:
import json
import pysal as ps
import urllib
import numpy as np
from urlparse import urlparse

#test cases for geointerfaces
import geopandas as gpd
import shapely.geometry as geom
import geojson as gj

Just extending on the prior use of `contiguity_from_geojson`, I thought it'd be cool to extend it to arbitrary interfaces that admit geointerfaces. This includes:

1. `pysal` file objects
2. `geopandas` dataframes/geoseries
3. json (local or remote, using `parse_geojson`)
4. iterables of `shapely` objects


This needs the `wconstructor` branch.

In [22]:
def parse_geojson(resource):
    gj = urlparse(resource)
    if not gj[1]:
        # either a serialized object or a local file
        try:
            return json.loads(gj[2])
        except:
            with open(gj[2]) as f:
                return json.load(f)    
    else:
        # remote url
        fs = urllib.urlopen(resource)
        return json.load(fs)
    return gj

In [23]:
def contiguity_from_GeoInterfaces(geoints, wtype = 'rook'):
    WTYPE = wtype.upper()
    if WTYPE not in ['QUEEN', 'ROOK']:
        raise ValueError("wtype must be 'QUEEN' or 'ROOK'")
    WTYPE =['QUEEN', 'ROOK'].index(WTYPE)+1
    if not np.all([hasattr(feature, '__geo_interface__') for feature in geoints]):
        try:
            feats = parse_geojson(geoints)['features']
            geoints = [ps.cg.asShape(feature['geometry']) for feature in feats]
        except:
            raise AttributeError("Iterable must admit a '__geo_interface__' method")
    geoints = [feature.__geo_interface__ for feature in geoints]
    if np.all([feature['type'] in ['Polygon', 'MultiPolygon'] for feature in geoints]):
        shpdict = {idx:ps.cg.asShape(poly) for idx, poly in enumerate(geoints)}
        pc = ps.cg.shapes.PolygonCollection(shpdict)
        neighs = ps.weights.Contiguity.ContiguityWeightsPolygons(pc)
    else:
        raise NotImplementedError('Contiguity Weights from objects with different dimensionality not supported')
    return ps.W(neighs.w)

In [24]:
path = ps.examples.get_path('columbus.shp')

In [25]:
A = contiguity_from_GeoInterfaces(ps.open(path))

setting bounding box for FeatureCollection


In [26]:
B = contiguity_from_GeoInterfaces(gpd.read_file(path).geometry)

setting bounding box for FeatureCollection


In [27]:
C = contiguity_from_GeoInterfaces('http://toae.org/pub/columbus.json')

setting bounding box for FeatureCollection


In [28]:
D = contiguity_from_GeoInterfaces([geom.shape(poly) for poly in ps.open(path)])

setting bounding box for FeatureCollection


In [29]:
A.neighbors == B.neighbors == C.neighbors == D.neighbors

True

I dunno how much more arbitrary you can get. If something implements the geointerface or comes close to being able to be coerced into it, we can construct spatial weights for it with serge's new `wconstructor`. 

### Playing around with a pysal-geopandas API

In [30]:
def lag_col(df, colname):
    return gpd.GeoSeries(ps.lag_spatial(df.W, df[colname]))

In [60]:
data = gpd.read_file(ps.examples.get_path('columbus.shp'))

Now that this is more arbitrary, it might make sense for `GeoPandas` dataframes to have some `W` attribute or method to construct `W` matrices. Here, I'll just assign.

In [61]:
data.W = contiguity_from_GeoInterfaces(data.geometry, wtype='QUEEN')

setting bounding box for FeatureCollection


In [62]:
data.columns

Index([u'AREA', u'COLUMBUS_', u'COLUMBUS_I', u'CP', u'CRIME', u'DISCBD', u'EW', u'HOVAL', u'INC', u'NEIG', u'NEIGNO', u'NSA', u'NSB', u'OPEN', u'PERIMETER', u'PLUMB', u'POLYID', u'THOUS', u'X', u'Y', u'geometry'], dtype='object')

In general, it might be a good idea to have a `.lag_column` method. toying with that here.

In [63]:
data['HOVAL_lag'] = lag_col(data, 'HOVAL')

In [64]:
data.lag_col = lambda x: lag_col(data, x)

Now to test how to use pysal on geopandas objects. 

In [143]:
Mo = ps.Moran(data['HOVAL'].values, data.W)

In [66]:
Mo.I

0.18009311431727287

In [67]:
d = data['HOVAL']

In [68]:
Mo2 = ps.Moran(np.array(d), data.W)

In [69]:
Mo.I == Mo2.I

True

In [70]:
Mo.p_norm == Mo2.p_norm

True

In [71]:
Mo.z2ss

16367.794631703126

In [72]:
Mo2.z2ss

16367.794631703126

In [87]:
BV = ps.Moran_BV(data['HOVAL'].values, data['CRIME'], data.W)

In [86]:
LMo = ps.Moran_Local(data['HOVAL'].values, data.W)

So, all moran calculations can be done simply by using the `.values` casting.

In [91]:
data.columns

Index([u'AREA', u'COLUMBUS_', u'COLUMBUS_I', u'CP', u'CRIME', u'DISCBD', u'EW', u'HOVAL', u'INC', u'NEIG', u'NEIGNO', u'NSA', u'NSB', u'OPEN', u'PERIMETER', u'PLUMB', u'POLYID', u'THOUS', u'X', u'Y', u'geometry', u'HOVAL_lag'], dtype='object')

In [132]:
reg = ps.spreg.OLS(data['HOVAL'].values.reshape(49,1), 
                   data[['INC', 'CRIME']].values, 
                   data.W, spat_diag=True, moran=True,
                   name_y = 'HOVAL', name_x = ['INC', 'CRIME'])

In [133]:
print reg.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :       HOVAL                Number of Observations:          49
Mean dependent var  :     38.4362                Number of Variables   :           3
S.D. dependent var  :     18.4661                Degrees of Freedom    :          46
R-squared           :      0.3495
Adjusted R-squared  :      0.3212
Sum squared residual:   10647.015                F-statistic           :     12.3582
Sigma-square        :     231.457                Prob(F-statistic)     :   5.064e-05
S.E. of regression  :      15.214                Log likelihood        :    -201.368
Sigma-square ML     :     217.286                Akaike info criterion :     408.735
S.E of regression ML:     14.7406                Schwarz criterion     :     414.411

-----------------------------------------------------------------------------

The `spreg` module also can be used, but we run into [the shape issue](https://github.com/pysal/pysal/issues/461) again.

In [137]:
pci = gpd.pd.read_csv(ps.examples.get_path('usjoin.csv'))
states = gpd.read_file(ps.examples.get_path('us48.shp'))

In [138]:
pci.columns

Index([u'Name', u'STATE_FIPS', u'1929', u'1930', u'1931', u'1932', u'1933', u'1934', u'1935', u'1936', u'1937', u'1938', u'1939', u'1940', u'1941', u'1942', u'1943', u'1944', u'1945', u'1946', u'1947', u'1948', u'1949', u'1950', u'1951', u'1952', u'1953', u'1954', u'1955', u'1956', u'1957', u'1958', u'1959', u'1960', u'1961', u'1962', u'1963', u'1964', u'1965', u'1966', u'1967', u'1968', u'1969', u'1970', u'1971', u'1972', u'1973', u'1974', u'1975', u'1976', u'1977', u'1978', u'1979', u'1980', u'1981', u'1982', u'1983', u'1984', u'1985', u'1986', u'1987', u'1988', u'1989', u'1990', u'1991', u'1992', u'1993', u'1994', u'1995', u'1996', u'1997', u'1998', u'1999', u'2000', u'2001', u'2002', u'2003', u'2004', u'2005', u'2006', u'2007', u'2008', u'2009'], dtype='object')

In [139]:
states.columns

Index([u'AREA', u'PERIMETER', u'STATE_', u'STATE_ABBR', u'STATE_FIPS', u'STATE_ID', u'STATE_NAME', u'SUB_REGION', u'geometry'], dtype='object')

In [140]:
new = gpd.pd.merge(states,pci, left_on='STATE_NAME', right_on='Name')

And, from here, you could just do whatever you need to do using `.values` and preexisting code to do any kind of markov model in `spatial_dynamics`, which means that a large fraction of use cases of the library are covered. 